# The CHOCOdetector pipeline

In [16]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
import pickle
import itertools
import pandas as pd
from pathlib import Path
from src.chocodetector import ChocoDetector
from src.utils import *

In [18]:
reference_filenames, reference_images = load_images(
    # dir="data/references", downsampling_factor=4
    dir="/Users/subnaulitus/Documents/EPFL/Master/MA2/EE-451/IAPR_39/project/data/references", downsampling_factor=4
)

In [19]:
training_filenames, training_images = load_images(
    # dir="data/train", downsampling_factor=4
    dir="/Users/subnaulitus/Documents/EPFL/Master/MA2/EE-451/IAPR_39/project/data/train", downsampling_factor=4
)

In [20]:
testing_filenames, testing_images = load_images(
    # dir="data/test", downsampling_factor=4,
    dir="/Users/subnaulitus/Documents/EPFL/Master/MA2/EE-451/IAPR_39/project/data/test", downsampling_factor=4,
)

In [21]:
model = ChocoDetector(
	sliding_window_size=64,
	sliding_window_stride=8,
	n_bins_histogram=16,
	heatmap_percentile=97.3,
	blob_min_sigma=3.0,
	blob_max_sigma=8.0,
	blob_thr=0.06,
	blob_avg_R=6,
	# classifier_checkpoint_path="./checkpoints/svc_chocolate_classifier.pkl", # 99% pca
	classifier_checkpoint_path="./checkpoints/svc_chocolate_classifier95.pkl", # 95% pca
	submission_template_df=pd.read_csv("/Users/subnaulitus/Documents/EPFL/Master/MA2/EE-451/IAPR_39/project/data/sample_submission.csv"),
	train_labels_df=pd.read_csv("/Users/subnaulitus/Documents/EPFL/Master/MA2/EE-451/IAPR_39/project/data/train.csv"),
	optimisation_mode=False,
)

model.load_data(
	ref_images_rgb=reference_images,
	ref_filenames=reference_filenames,
	train_images_rgb=training_images,
	train_filenames=training_filenames,
	test_images_rgb=testing_images,
	test_filenames=testing_filenames,
)
    
general_back_submission = model.run()

DEBUG - Optimisation is False: (180, 13)
DEBUG - Running pipeline on 180 samples


100%|██████████| 180/180 [04:58<00:00,  1.66s/img]


In [22]:
clean_back_submission = pd.read_csv('./clean_background_result.csv', index_col='id')

In [23]:
# Merge the dataframes, replacing rows in general_back_submission with matching IDs from clean_back_submission
for idx in clean_back_submission.index:
    general_back_submission.loc[idx] = clean_back_submission.loc[idx]

In [24]:
final_submission = general_back_submission.copy(deep=True)
final_submission.to_csv('final_submission.csv', index=True)